In [5]:
import pandas as pd
import numpy as np
import joblib
import requests
from datetime import datetime, timezone, timedelta
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error

# ✅ Load the dataset
df = pd.read_csv("C:/Users/incha/Downloads/flood.csv")

# ✅ Select relevant features
features = [
    "MonsoonIntensity", "TopographyDrainage", "RiverManagement",
    "Deforestation", "Urbanization", "ClimateChange", "DamsQuality",
    "Siltation", "AgriculturalPractices", "Encroachments",
    "IneffectiveDisasterPreparedness", "DrainageSystems",
    "CoastalVulnerability", "Landslides", "Watersheds",
    "DeterioratingInfrastructure", "PopulationScore", "WetlandLoss",
    "InadequatePlanning", "PoliticalFactors"
]
target = "FloodProbability"

X = df[features]
y = df[target]

# ✅ If FloodProbability is binary (0 or 1), convert to int
if y.nunique() == 2:
    y = y.astype(int)
    is_classification = True
else:
    is_classification = False

# ✅ Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ✅ Choose Model Type Based on Target Variable
if is_classification:
    model = RandomForestClassifier(n_estimators=200, random_state=42)
else:
    model = RandomForestRegressor(n_estimators=200, random_state=42)

# ✅ Train Model
model.fit(X_train_scaled, y_train)

# ✅ Save Model & Scaler
joblib.dump(model, "flood_model.pkl")
joblib.dump(scaler, "scaler.pkl")

# ✅ Evaluate Model
if is_classification:
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Classification Accuracy: {accuracy * 100:.2f}%")
else:
    y_pred = model.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Model Regression Mean Absolute Error: {mae:.4f}")


Model Regression Mean Absolute Error: 0.0203


In [6]:
import joblib

# ✅ Load trained model and scaler
model = joblib.load("D:/flood_prediction/flood_model.pkl")
scaler = joblib.load("D:/flood_prediction/scaler.pkl")

def get_weather_data(api_key, location):
    """Fetch real-time weather data."""
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
    response = requests.get(url).json()

    if "main" in response:
        return {
            "MonsoonIntensity": np.random.uniform(1, 10),  # Simulated values
            "TopographyDrainage": np.random.uniform(1, 10),
            "RiverManagement": np.random.uniform(1, 10),
            "Deforestation": np.random.uniform(1, 10),
            "Urbanization": np.random.uniform(1, 10),
            "ClimateChange": np.random.uniform(1, 10),
            "DamsQuality": np.random.uniform(1, 10),
            "Siltation": np.random.uniform(1, 10),
            "AgriculturalPractices": np.random.uniform(1, 10),
            "Encroachments": np.random.uniform(1, 10),
            "IneffectiveDisasterPreparedness": np.random.uniform(1, 10),
            "DrainageSystems": np.random.uniform(1, 10),
            "CoastalVulnerability": np.random.uniform(1, 10),
            "Landslides": np.random.uniform(1, 10),
            "Watersheds": np.random.uniform(1, 10),
            "DeterioratingInfrastructure": np.random.uniform(1, 10),
            "PopulationScore": np.random.uniform(1, 10),
            "WetlandLoss": np.random.uniform(1, 10),
            "InadequatePlanning": np.random.uniform(1, 10),
            "PoliticalFactors": np.random.uniform(1, 10),
        }
    return None

def predict_flood(api_key):
    """Predict flood risk for today & next 10 days."""
    location = input("Enter city name: ")
    weather_data = get_weather_data(api_key, location)

    if weather_data:
        print(f"\n--- Weather Data Fetched ---")
        print(f"City: {location}")
        today_date = datetime.now().strftime('%Y-%m-%d')
        print(f"Date: {today_date}")

        # Convert weather data to DataFrame and scale it
        weather_df = pd.DataFrame([weather_data])
        weather_scaled = scaler.transform(weather_df)

        # Predict flood probability
        today_prediction = model.predict(weather_scaled)[0]

        # Interpret today's prediction
        if today_prediction == 1:
            today_flood_risk = "High Risk Flood Area"
            today_alert = "⚠ Extreme Alert: Severe Flooding Expected!"
        else:
            today_flood_risk = "Flood Free Area"
            today_alert = "✅ No Immediate Flood Threat"

        print(f"\n--- Flood Prediction for Today ({today_date}) ---")
        print(f"Flood Risk Level: {today_flood_risk}")
        print(today_alert)

        # Predict flood risk for the next 10 days
        print("\n--- Flood Prediction for Next 10 Days ---")
        for i in range(1, 11):
            future_date = (datetime.now() + timedelta(days=i)).strftime('%Y-%m-%d')
            weather_data = get_weather_data(api_key, location)

            if weather_data:
                weather_df = pd.DataFrame([weather_data])
                weather_scaled = scaler.transform(weather_df)
                prediction = model.predict(weather_scaled)[0]

                if prediction == 1:
                    flood_risk = "High Risk Flood Area"
                    alert = "⚠ Extreme Alert: Severe Flooding Expected!"
                else:
                    flood_risk = "Flood Free Area"
                    alert = "✅ No Immediate Flood Threat"

                print(f"\nPredicted Flood for {future_date}:")
                print(f"Flood Risk Level: {flood_risk}")
                print(alert)
    else:
        print("❌ Failed to retrieve weather data.")




In [7]:
# ✅ Run Prediction
API_KEY = "e3a90990b56034cdb755847b6439abae"
predict_flood(API_KEY)


--- Weather Data Fetched ---
City: mysuru
Date: 2025-02-28

--- Flood Prediction for Today (2025-02-28) ---
Flood Risk Level: Flood Free Area
✅ No Immediate Flood Threat

--- Flood Prediction for Next 10 Days ---

Predicted Flood for 2025-03-01:
Flood Risk Level: Flood Free Area
✅ No Immediate Flood Threat

Predicted Flood for 2025-03-02:
Flood Risk Level: Flood Free Area
✅ No Immediate Flood Threat

Predicted Flood for 2025-03-03:
Flood Risk Level: Flood Free Area
✅ No Immediate Flood Threat

Predicted Flood for 2025-03-04:
Flood Risk Level: Flood Free Area
✅ No Immediate Flood Threat

Predicted Flood for 2025-03-05:
Flood Risk Level: Flood Free Area
✅ No Immediate Flood Threat

Predicted Flood for 2025-03-06:
Flood Risk Level: Flood Free Area
✅ No Immediate Flood Threat

Predicted Flood for 2025-03-07:
Flood Risk Level: Flood Free Area
✅ No Immediate Flood Threat

Predicted Flood for 2025-03-08:
Flood Risk Level: Flood Free Area
✅ No Immediate Flood Threat

Predicted Flood for 2025-